If you dont mind lets look at another example using Biddy et al dataset. 
Here we want to inteprolate between failed to reprogrammed cells and detect master regulators in this process.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import time
import scanpy as sc


In [ ]:
adata = sc.read('CellTagging_adata_preprocessed.h5ad')
adata.obs.state_info.value_counts()
sc.pp.filter_cells(adata, min_genes=100)
sc.pp.filter_genes(adata, min_cells=200)
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2000)
adata = adata[:, adata.var['highly_variable']]
data=pd.DataFrame((adata.X))
adata = adata[((adata.obs.state_info == "Reprogrammed") | (adata.obs.state_info == "Failed"))]
data=pd.DataFrame(adata.X.todense())
data = data.to_numpy(dtype=np.float64)

In [ ]:
# Divide data into train and test sets
train_data = data[:3000]
test_data = data[3000:]

In [ ]:
laten_size=45
layer1=250
genes_number=2000
# Convert data to PyTorch tensors and create data loaders
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_loader = DataLoader(TensorDataset(train_data), batch_size=500, shuffle=True)
test_loader = DataLoader(TensorDataset(test_data), batch_size=500, shuffle=True)

# Define the variational autoencoder model
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(genes_number, layer1),
            nn.ReLU(),
            nn.Dropout(p=0.15),
            nn.Linear(layer1, laten_size),
        )
        self.decoder = nn.Sequential(
            nn.Linear(laten_size, layer1),
            nn.ReLU(),
            nn.Linear(layer1, genes_number),
        )
        self.mu = nn.Linear(laten_size, laten_size)
        self.log_var = nn.Linear(laten_size, laten_size)

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        x = self.encoder(x)
        mu = self.mu(x)
        log_var = self.log_var(x)
        z = self.reparameterize(mu, log_var)
        x = self.decoder(z)
        return x, mu, log_var

    def sample(self, num_samples):
        z = torch.randn(num_samples, laten_size)
        return self.decoder(z)

# Initialize the model and optimizer
model = VAE()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
def vae_loss(x, x_recon, mu, log_var):
    recon_loss = nn.functional.mse_loss(x_recon, x, reduction='sum')
    kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return recon_loss + kl_div

# Train the model
train_losses = []
test_losses = []
num_epochs = 250
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        x = batch[0]
        x_recon, mu, log_var = model(x)
        loss = vae_loss(x, x_recon, mu, log_var)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_losses.append(train_loss / len(train_data))

    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            x = batch[0]
            x_recon, mu, log_var = model(x)
            loss = vae_loss(x, x_recon, mu, log_var)
            test_loss += loss.item()
        test_losses.append(test_loss / len(test_data))

    print(f"Epoch {epoch+1}, Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}")

# Plot the training and validation loss
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.legend()
plt.show()


# Convert the data to a PyTorch tensor
data_tensor = torch.tensor(data).float()

# Compute the latent layer for the data
with torch.no_grad():
    model.eval()
    latent_layer = model.encoder(data_tensor)
    

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

latent_layer = (latent_layer - latent_layer.mean())/(latent_layer.std())
s_curve = latent_layer


adata.obs.columns
label_encoder = LabelEncoder()
numeric_labels = label_encoder.fit_transform(adata.obs.state_info)
import umap.umap_ as umap
umap_embedding = umap.UMAP(n_neighbors=50, min_dist=0.8, random_state=42).fit_transform(latent_layer)
plt.scatter(umap_embedding[:, 0], umap_embedding[:, 1], c=numeric_labels, s=5, cmap='viridis')
plt.savefig("Good umap of failed reporogrammed.pdf") 
plt.show()
dataset = torch.Tensor(latent_layer).float()

Here I used the sigmoid function but you can choose other functions for variance scheduling. You can find them in the first Jupyter notebook

In [ ]:
num_steps = 1000 
#Develop the beta of each step
betas = torch.linspace(-6,6,num_steps)
betas = torch.sigmoid(betas)*(0.5e-2 - 1e-5) + 1e-5
alphas = 1-betas
alphas_prod = torch.cumprod(alphas,0)
alphas_prod_p = torch.cat([torch.tensor([1]).float(),alphas_prod[:-1]],0)
alphas_bar_sqrt = torch.sqrt(alphas_prod)
one_minus_alphas_bar_log = torch.log(1 - alphas_prod)
one_minus_alphas_bar_sqrt = torch.sqrt(1 - alphas_prod)

Forward steps in DDM:

In [ ]:
def q_x(x_0,t):
    """You can get x[t] at any time t based on x[0]"""
    noise = torch.randn_like(x_0)
    alphas_t = alphas_bar_sqrt[t]
    alphas_1_m_t = one_minus_alphas_bar_sqrt[t]
    return (alphas_t * x_0 + alphas_1_m_t * noise) #Add noise based on x[0]

In [ ]:
# choose how many step you want to see
num_shows = 20
# choose your figure size
fig,axs = plt.subplots(,,figsize=(20,5))
for i in range(num_shows):
    print(i)
    j = i//10
    k = i%10
    q_i = q_x(dataset,torch.tensor([i*num_steps//num_shows]))      #Generate sample data at time t
    umap_emb = umap.UMAP(n_neighbors=80, min_dist=0.3).fit_transform(q_i)
    axs[j,k].scatter(umap_emb[:, 0], umap_emb[:, 1], c=numeric_labels, s=5, cmap='viridis');

Neural network architecture to approximate the conditioned probability distributions in the reverse diffusion proces.

In [ ]:
class MLPDiffusion(nn.Module):
    def __init__(self,n_steps, num_units=1200):
        super(MLPDiffusion,self).__init__()
        
        self.linears = nn.ModuleList(
            [
                nn.Linear(laten_size,num_units),
                nn.ReLU(),
                nn.Linear(num_units,num_units),
                nn.ReLU(),
                nn.Linear(num_units,num_units),
                nn.ReLU(),
                nn.Linear(num_units,laten_size),
            ]
        )
        self.step_embeddings = nn.ModuleList(
            [
                nn.Embedding(n_steps,num_units),
                nn.Embedding(n_steps,num_units),
                nn.Embedding(n_steps,num_units),
            ]
        )
    def forward(self,x,t):
#         x = x_0
        for idx,embedding_layer in enumerate(self.step_embeddings):
            t_embedding = embedding_layer(t)
            x = self.linears[2*idx](x)
            x += t_embedding
            x = self.linears[2*idx+1](x)
            
        x = self.linears[-1](x)
        
        return x


In [ ]:
def diffusion_loss_fn(model,x_0,alphas_bar_sqrt,one_minus_alphas_bar_sqrt,n_steps):
    batch_size = x_0.shape[0]
    ##Generate random time t for a batchsize sample
    t = torch.randint(0,n_steps,size=(batch_size//2,), device=x_0.device)
    t = torch.cat([t,n_steps-1-t],dim=0)
    t = t.unsqueeze(-1)
    #Coefficient of x0
    a = alphas_bar_sqrt[t]
    #Coefficient of eps
    aml = one_minus_alphas_bar_sqrt[t]
    #Generate random noise eps
    e = torch.randn_like(x_0, device=x_0.device)
    #Construct the input of the model
    x = (x_0*a+e*aml).to(device)
    #Send into the model to get the predicted value of random noise at time t
    output = model(x, t.squeeze(-1))
    #Compute error along with real noise, average
    return (e - output).square().mean()

In [ ]:
def p_sample_loop(model, shape, n_steps, betas, one_minus_alphas_bar_sqrt):
    """Restore x[T-1], x[T-2]|...x[0] from x[T]"""
    cur_x = torch.randn(shape).to(device)
    x_seq = [cur_x]
    for i in reversed(range(n_steps)):
        cur_x = p_sample(model, cur_x, i, betas, one_minus_alphas_bar_sqrt).to(device)
        x_seq.append(cur_x)
    return x_seq

def p_sample(model, x, t, betas, one_minus_alphas_bar_sqrt):
    device="cpu"
    t = torch.tensor([t]).to(device)
    betas = betas.to(device)
    one_minus_alphas_bar_sqrt = one_minus_alphas_bar_sqrt.to(device)
    coeff = (betas[t] / one_minus_alphas_bar_sqrt[t]).to(device)
    eps_theta = model(x, t).to(device)
    mean = (1 / (1 - betas[t]).sqrt()) * (x - (coeff * eps_theta)).to(device)
    z = torch.randn_like(x).to(device)
    sigma_t = betas[t].sqrt().to(device)
    sample = mean + sigma_t * z
    return sample

In [ ]:
class EMA():
    def __init__(self, mu=0.001):
        self.mu = mu
        self.shadow = {}
    def register(self,name,val):
        self.shadow[name] = val.clone()
    def __call__(self,name,x):
        assert name in self.shadow
        new_average = self.mu * x + (1.0-self.mu)*self.shadow[name]
        self.shadow[name] = new_average.clone()
        return new_average

device="cuda"
total_loss=[]
print('Training model...')
batch_size = 1000
dataloader = torch.utils.data.DataLoader(dataset[:], batch_size=batch_size, shuffle=True)
num_epoch = 1500
plt.rc('text',color='blue')
model2 = MLPDiffusion(num_steps) #The output dimension is 2, the input is x and step
model2.to(device)
model2 = torch.nn.DataParallel(model2).to(device)
alphas_bar_sqrt = alphas_bar_sqrt.to(device)
one_minus_alphas_bar_sqrt = one_minus_alphas_bar_sqrt.to(device)
optimizer = torch.optim.Adam(model2.parameters(),lr=1e-3)

for t in range(num_epoch):
    print("The values are: {} and {}".format(loss, t))
    for idx,batch_x in enumerate(dataloader):
        batch_x = batch_x.to(device)
        loss = diffusion_loss_fn(model2, batch_x, alphas_bar_sqrt, one_minus_alphas_bar_sqrt, num_steps)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model2.parameters(),1.)
        optimizer.step()      

In [ ]:
PATH = "Morris.pth"
# Save
torch.save(model2.state_dict(), PATH)
# Load
device = torch.device('cpu')
model4 = MLPDiffusion(1000)

# original saved file with DataParallel
state_dict = torch.load("Morris.pth")
# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
# load params
model4.load_state_dict(new_state_dict)

In [ ]:
condition=pd.DataFrame(adata.obs.state_info)
condition=condition.reset_index()
condition = condition.drop(condition.columns[0], axis=1)

merged = pd.concat([condition, pd.DataFrame(latent_layer)],axis=1)
reprog = merged[(merged.state_info == "Reprogrammed")]
failed = merged[(merged.state_info == "Failed")]

reprog_diff_latent = q_x(torch.tensor(reprog.iloc[:,1:].values), torch.tensor([num_steps-1]))
failed_diff_latent = q_x(torch.tensor(failed.iloc[:,1:].values), torch.tensor([num_steps-1]))
reprog_diff_latent_m=torch.tensor(reprog_diff_latent.mean(axis=0))
failed_diff_latent_m=torch.tensor(failed_diff_latent.mean(axis=0))

In [ ]:
umap_emb = umap.UMAP(n_neighbors=80, min_dist=0.3).fit_transform(failed_diff_latent)
plt.scatter(umap_emb[:, 0], umap_emb[:, 1], s=5, cmap='viridis');

In [ ]:
alpha = torch.tensor(np.linspace(0, 1, 5000, dtype=np.float32)).to(device)
intp = reprog_diff_latent_m* (1 - alpha[:, None]) + failed_diff_latent_m * alpha[:, None]

In [ ]:
def p_sample_loop2(model, shape, n_steps, betas, one_minus_alphas_bar_sqrt):
    """Restore x[T-1], x[T-2]|...x[0] from x[T]"""
    cur_x = intp.to(device)
    x_seq = [cur_x]
    for i in reversed(range(n_steps)):
        cur_x = p_sample(model, cur_x, i, betas, one_minus_alphas_bar_sqrt)
        x_seq.append(cur_x)
    return x_seq

In [ ]:
x_seq_zero = p_sample_loop2(model4, intp.shape, num_steps, betas, one_minus_alphas_bar_sqrt)

In [ ]:
with torch.no_grad():
    decod_intrp = model.decoder(x_seq_zero[1000])
column_names = list(adata.var_names)  # List of column names from the "data" DataFrame

You can ask me why are you computing velocity at all???? I believe looking at gene expression alone is not enough. 
In classical mechanics we have seen the benfit of using concpets such as velocity, momentum, acceleration, and jerk (rate at which an object's acceleration changes). I believe, when a model is capable of generating realistic and novel data, it indicates that it has learned the appropriate embedding. Also, Bengio et al. have demonstrated that causal models tend to adapt more quickly to interventions compared to predictive models. In this scenario, as we interpolate using a neural network, it is expected that key genes will undergo more rapid changes and exhibit higher velocity. I think it's not a bad idea to consider additional factors beyond gene expression, which might help us in better understanding the underlying phenomena.

In [ ]:
# list of top 30 genes with the highest velocity
velocities = decod_intrp.diff()
top_cols = velocities.apply(lambda x: x.nlargest(30).index.tolist(), axis=1)
top_cols.to_csv("velocity_genes.csv")


In [ ]:
# We can also compute the average velocity. Here I consider changes in values over each 10-step interval. 
# The resulting differences are then divided by 10 to obtain the velocity values.
decod_intrp.index = pd.RangeIndex(start=0, stop=5000)
decod_intrp.index = pd.to_datetime(decod_intrp.index, unit='s')
df_resampled = decod_intrp.resample('10S').mean()
velocities2 = df_resampled.diff() / (10)
top_cols2 = velocities2.apply(lambda x: x.nlargest(50).index.tolist(), axis=1)
